<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/eda/AI_risks_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets

* AI Incident Database <br>
https://www.kaggle.com/datasets/konradb/ai-incident-database
* Government AI Readiness Index <br>
https://www.statista.com/statistics/1231685/worldwide-government-artificial-intelligence-readiness-index/ <br>
https://www.statista.com/statistics/1231719/eastern-europe-government-artificial-intelligence-readiness-index/
* AI Ethics Guidelines Global Inventory <br>
https://www.statista.com/statistics/1286900/ai-ethics-principles-by-organization-type/


# Questions

1. What are main AI failure types?
2. Failure types evolution through time.
3. Failure types by domain/company/country/demographics/models.
4. Consequences of AI failures
5. How these AI risks map onto existing ethical guidelines and regulations?
6. What are the mitigation strategies that are most effective?
7. How to prevent those risks?



In [1]:
!pip install -U -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
incident_dataset = load_dataset("vitaliy-sharandin/ai-incidents")
incident_dataset = incident_dataset['train'].to_pandas()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]